<a href="https://colab.research.google.com/github/OliBomby/Mapperatorinator/blob/main/colab/mapperatorinator_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beatmap Generation with Mapperatorinator

This notebook is an interactive demo of an osu! beatmap generation model created by OliBomby. This model is capable of generating hit objects, hitsounds, timing, kiai times, and SVs for all 4 gamemodes. You can upload a beatmap to give to the model as additional context or remap parts of the beatmap.

### Instructions for running:

* Read and accept the rules regarding using this tool by clicking the checkbox.
* __Execute each cell in order__. Press ▶️ on the left of each cell to execute the cell.
* __Setup Environment__: run the first cell to clone the repository and install the required dependencies. You only need to run this cell once per session.
* __Upload Audio__: specify the path to a .mp3 or .ogg file.
* __Upload Beatmap__: optionally specify the path to a beatmap .osu file. You can find these files in stable by using File > Open Song Folder, or in lazer by using File > Edit Externally.
* __Configure__: choose your generation parameters to control the style of the generated beatmap.
* Generate the beatmap using the __Generate Beatmap__ cell. (it may take a few minutes depending on the length of the song)


In [ ]:
#@title Setup Environment { display-mode: "form" }
#@markdown ### Use this tool responsibly. Always disclose the use of AI in your beatmaps. Accept the rules and run this cell.
i_accept_the_rules = False # @param {type:"boolean"}

assert i_accept_the_rules, "Read and accept the rules first!"

!git clone https://github.com/OliBomby/Mapperatorinator.git
%cd Mapperatorinator

!pip install transformers==4.53.3
!pip install hydra-core nnaudio
!pip install slider git+https://github.com/OliBomby/slider.git@gedagedigedagedaoh

import os
from IPython.display import FileLink, display
from hydra import compose, initialize_config_dir
from osuT5.osuT5.event import ContextType
from inference import main

output_path = "output"
input_audio = ""
input_beatmap = ""

In [ ]:
#@title Upload Audio { display-mode: "form" }
#@markdown Specify the path to your audio file. This is the song to generate a beatmap for. Please use a .mp3 or .ogg file.
audio_file_path = "" # @param {type:"string"}

def set_audio(path):
    if not path:
        print('Please specify an audio file path.')
        return ""
    if not os.path.exists(path):
        print(f'File not found: {path}')
        return ""
    if not path.endswith('.mp3') and not path.endswith('.ogg'):
        print('Invalid file format. Please use a .mp3 or .ogg file.')
        return ""
    return path

input_audio = set_audio(audio_file_path)

In [ ]:
#@title (Optional) Upload Beatmap { display-mode: "form" }
#@markdown This step is required if you want to use `in_context` or `add_to_beatmap` to provide additional info to the model.
#@markdown It will also fill in any missing metadata and unknown values in the configuration using info of the reference beatmap.
#@markdown Please specify a **.osu** file path. You can find the .osu file in the song folder in stable or by using File > Edit Externally in lazer.
use_reference_beatmap = False # @param {type:"boolean"}
beatmap_file_path = "" # @param {type:"string"}

def set_beatmap(path):
    if not path:
        return ""
    if not os.path.exists(path):
        print(f'File not found: {path}')
        return ""
    if not path.endswith('.osu'):
        print('Invalid file format. Please use a .osu file.\nIn stable you can find the .osu file in the song folder (File > Open Song Folder).\nIn lazer you can find the .osu file by using File > Edit Externally.')
        return ""
    return path

if use_reference_beatmap:
    input_beatmap = set_beatmap(beatmap_file_path)
else:
    input_beatmap = ""

In [ ]:
#@title Configure and Generate Beatmap { display-mode: "form" }

#@markdown #### You can input -1 to leave the value unknown.
#@markdown ---
#@markdown This is the AI model to use. V30 is the most accurate model, but it does not support other gamemodes, year, descriptors, or in_context.
model = "Mapperatorinator V30" # @param ["Mapperatorinator V29", "Mapperatorinator V30", "Mapperatorinator V31"]
#@markdown This is the game mode to generate a beatmap for.
gamemode = "standard" # @param ["standard", "taiko", "catch the beat", "mania"]
#@markdown This is the Star Rating you want your beatmap to be. It might deviate from this number depending on the song intensity and other configuration.
difficulty = 5 # @param {type:"number"}
#@markdown This is the user ID of the ranked mapper to imitate for mapping style. You can find this in the URL of the mapper's profile.
mapper_id = -1 # @param {type:"integer"}
#@markdown This is the year you want the beatmap to be from. It should be in the range of 2007 to 2023.
year = 2023 # @param {type:"integer"}
#@markdown Optional HuggingFace repository ID of a LoRA to be applied to the current model.
lora_path = "" # @param {type:"string"}
#@markdown This is whether you want the beatmap to be hitsounded. This works only for mania mode.
hitsounded = True # @param {type:"boolean"}
#@markdown These are the standard difficulty parameters for the beatmap HP, OD, AR, and CS. These are the same as the ones in the editor.
hp_drain_rate = 5 # @param {type:"number"}
circle_size = 4 # @param {type:"number"}
overall_difficulty = 9 # @param {type:"number"}
approach_rate = 8 # @param {type:"number"}
slider_multiplier = 1.4 # @param {type:"slider", min:0.4, max:3.6, step:0.1}
slider_tick_rate = 1 # @param {type:"number"}
#@markdown This is the number of keys for the mania beatmap. This works only for mania mode.
keycount = 4 # @param {type:"slider", min:1, max:18, step:1}
#@markdown This is the ratio of hold notes to circles in the beatmap. It should be in the range [0,1]. This works only for mania mode.
hold_note_ratio = -1 # @param {type:"number"}
#@markdown This is the ratio of scroll speed changes to the number of notes. It should be in the range [0,1]. This works only for mania and taiko modes.
scroll_speed_ratio = -1 # @param {type:"number"}
#@markdown These descriptors of the beatmap. Descriptors are used to describe the style of the beatmap. All available descriptors can be found [here](https://osu.ppy.sh/wiki/en/Beatmap/Beatmap_tags).
descriptor_1 = '' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
descriptor_2 = '' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
descriptor_3 = '' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
#@markdown These are negative descriptors of the beatmap. Negative descriptors are used to describe what the beatmap should not have. These work only when `cfg_scale` is greater than 1.
negative_descriptor_1 = '' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
negative_descriptor_2 = '' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
negative_descriptor_3 = '' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
#@markdown ---
#@markdown If true, the generated beatmap will be exported as a .osz file. Otherwise, it will be exported as a .osu file.
export_osz = False # @param {type:"boolean"}
#@markdown If true, the generated beatmap will be added to the reference beatmap and the reference beatmap will be modified instead of creating a new beatmap. It will also continue any hit objects before the start time in the reference beatmap.
add_to_beatmap = False # @param {type:"boolean"}
#@markdown This is the start time of the beatmap in milliseconds. Use this to constrain the generation to a specific part of the song.
start_time = -1 # @param {type:"integer"}
#@markdown This is the end time of the beatmap in milliseconds. Use this to constrain the generation to a specific part of the song.
end_time = -1 # @param {type:"integer"}
#@markdown This is which additional information to give to the model:
#@markdown - TIMING: Give timing points to the model. This will skip the timing point generation step.
#@markdown - KIAI: Give kiai times to the model. This will skip the kiai time generation step.
#@markdown - MAP: Give hit objects to the model. This will skip the hit object generation step.
#@markdown - GD: Give hit objects of another difficulty in the same mapset to the model (can be a different game mode). It will improve the rhythm accuracy and consistency of the generated beatmap without copying the reference beatmap.
#@markdown - NO_HS: Give hit objects without hitsounds to the model. This will copy the hit objects of the reference beatmap and only add hitsounds to them.
in_context = "[NONE]" # @param ["[NONE]", "[TIMING]", "[TIMING,KIAI]", "[TIMING,KIAI,MAP]", "[GD,TIMING,KIAI]", "[NO_HS,TIMING,KIAI]"]
#@markdown This is the output type of the beatmap. You can choose to either generate everything or only generate timing points.
output_type = "[MAP]" # @param ["[MAP]", "[TIMING,KIAI,MAP,SV]", "[TIMING]"]
#@markdown This is the scale of the classifier-free guidance. A higher scale will make the model more likely to follow the descriptors and mapper style. A high `cfg_scale` or certain combinations of settings can produce unexpected results, so use it with caution.
cfg_scale = 1 # @param {type:"slider", min:1, max:5, step:0.1}
#@markdown This is the temperature of the sampling. A lower temperature will make the model more conservative and less creative. I only recommend lowering this slightly or when using `add_to_beatmap` and generating small sections.
temperature = 1 # @param {type:"slider", min:0, max:1, step:0.01}
#@markdown This is the random seed. Change this to sample a different beatmap with the same settings.
seed = -1 # @param {type:"integer"}
#@markdown ---
#@markdown If true, uses a slow and accurate timing generator. This will make the generation slower, but the timing will be more accurate.
#@markdown This is the leniency of the normal timing generator. It will allow the timing ticks to deviate from the real timing by this many milliseconds. A higher value will result in less timing points.
timing_leniency = 20 # @param {type:"slider", min:0, max:100, step:1}
super_timing = False # @param {type:"boolean"}
#@markdown This is the number of beams for beam search for the super timing generator. Higher values will result in slightly more accurate timing at the cost of speed.
timer_num_beams = 2 # @param {type:"slider", min:1, max:16, step:1}
#@markdown This is the number of iterations for the super timing generator. Higher values will result in slightly more accurate timing at the cost of speed.
timer_iterations = 20 # @param {type:"slider", min:1, max:100, step:1}
#@markdown This is the certainty threshold requirement for BPM changes in the super timing generator. Higher values will result in less BPM changes.
timer_bpm_threshold = 0.1 # @param {type:"slider", min:0, max:1, step:0.1}
#@markdown ---

# Get actual parameters
a_config = model.split(' ')[-1].lower()
a_gamemode = ["standard", "taiko", "catch the beat", "mania"].index(gamemode)
a_difficulty = None if difficulty == -1 else difficulty
a_mapper_id = None if mapper_id == -1 else mapper_id
a_year = None if year == -1 else year
a_lora_path = None if lora_path == "" else lora_path
a_hp_drain_rate = None if hp_drain_rate == -1 else hp_drain_rate
a_circle_size = None if circle_size == -1 else circle_size
a_overall_difficulty = None if overall_difficulty == -1 else overall_difficulty
a_approach_rate = None if approach_rate == -1 else approach_rate
a_slider_multiplier = None if slider_multiplier == -1 else slider_multiplier
a_slider_tick_rate = None if slider_tick_rate == -1 else slider_tick_rate
a_hold_note_ratio = None if hold_note_ratio == -1 else hold_note_ratio
a_scroll_speed_ratio = None if scroll_speed_ratio == -1 else scroll_speed_ratio
descriptors = [d for d in [descriptor_1, descriptor_2, descriptor_3] if d != '']
negative_descriptors = [d for d in [negative_descriptor_1, negative_descriptor_2, negative_descriptor_3] if d != '']

a_start_time = None if start_time == -1 else start_time
a_end_time = None if end_time == -1 else end_time
a_in_context = [ContextType(c.lower()) for c in in_context[1:-1].split(',')]
a_output_type = [ContextType(c.lower()) for c in output_type[1:-1].split(',')]
a_seed = None if seed == -1 else seed

# Validate stuff
if any(c in a_in_context for c in [ContextType.TIMING, ContextType.KIAI, ContextType.MAP, ContextType.SV, ContextType.GD, ContextType.NO_HS]) or add_to_beatmap:
    assert os.path.exists(input_beatmap), "Please specify a reference beatmap."
assert os.path.exists(input_audio), "Please specify an audio file."
if a_config == "v30":
    assert a_gamemode == 0, "V30 only supports standard mode."
    if any(c in a_in_context for c in [ContextType.KIAI, ContextType.MAP, ContextType.SV]):
        print("WARNING: V30 does not support KIAI, MAP, or SV in_context, ignoring.")
    if output_type != "[MAP]":
        print("WARNING: V30 only supports [MAP] output type, setting output type to [MAP].")
        a_output_type = [ContextType.MAP]
    if len(descriptors) != 0 and len(negative_descriptors) != 0:
        print("WARNING: V30 does not support descriptors or negative descriptors, ignoring.")
    if super_timing:
        print("WARNING: V30 does not fully support super timing, generation will be VERY slow.")

# Create config
with initialize_config_dir(version_base="1.1", config_dir=os.path.join(os.getcwd(), "configs/inference")):
    conf = compose(config_name=a_config)

# Do inference
conf.audio_path = input_audio
conf.output_path = output_path
conf.beatmap_path = input_beatmap
conf.gamemode = a_gamemode
conf.difficulty = a_difficulty
conf.mapper_id = a_mapper_id
conf.year = a_year
conf.lora_path = a_lora_path
conf.hitsounded = hitsounded
conf.hp_drain_rate = a_hp_drain_rate
conf.circle_size = a_circle_size
conf.overall_difficulty = a_overall_difficulty
conf.approach_rate = a_approach_rate
conf.slider_multiplier = a_slider_multiplier
conf.slider_tick_rate = a_slider_tick_rate
conf.keycount = keycount
conf.hold_note_ratio = a_hold_note_ratio
conf.scroll_speed_ratio = a_scroll_speed_ratio
conf.descriptors = descriptors
conf.negative_descriptors = negative_descriptors
conf.export_osz = export_osz
conf.add_to_beatmap = add_to_beatmap
conf.start_time = a_start_time
conf.end_time = a_end_time
conf.in_context = a_in_context
conf.output_type = a_output_type
conf.cfg_scale = cfg_scale
conf.temperature = temperature
conf.seed = a_seed
conf.timing_leniency = timing_leniency
conf.super_timing = super_timing
conf.timer_num_beams = timer_num_beams
conf.timer_iterations = timer_iterations
conf.timer_bpm_threshold = timer_bpm_threshold

_, result_path, osz_path = main(conf)

if osz_path is not None:
    result_path = osz_path

if conf.add_to_beatmap:
    print(f"Generated beatmap saved at: {result_path}")
    display(FileLink(result_path))
else:
    print(f"Generated beatmap saved at: {result_path}")
    display(FileLink(result_path))


Using CUDA for inference (auto-selected).
Random seed: 20467
Using default bpm 120
Using default offset 0
Using default title Unknown Title
Using default artist Unknown Artist
Using default source 
Using default preview_time -1


/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to 

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Model loaded: OliBomby/Mapperatorinator-v29.1 on device cuda


tokenizer.pkl:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

model_ema.pkl:   0%|          | 0.00/534M [00:00<?, ?B/s]

Generating timing


 30%|███       | 6/20 [26:26<59:41, 255.85s/it]  